In [35]:
import pandas as pd
from datetime import datetime
import requests
import time 
import json
from requests_html import HTMLSession
from PIL import Image
from io import StringIO, BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output

%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [3]:
! pip install --upgrade selenium

In [16]:
!pip install jupyternotify

In [36]:
session = HTMLSession()

In [38]:
%%notify
start = time.time()
pages = 100
single_page_urls = []
page_cnt, url_cnt = 0, 0
category = 'electronics'
for p in range(1, pages+1):
    bikroy_url = f'https://bikroy.com/en/ads/bangladesh/{category}?sort=date&order=desc&buy_now=0&urgent=0&page={p}'
    print(bikroy_url)
    r = session.get('http://localhost:8055/render.html', params={'url': bikroy_url, 'wait': 1})

    page_cnt += 1
    for i in range (1, 28):
        try:
#             single_url = driver.find_element(by=By.XPATH, value=f'//*[@id="app-wrapper"]/div[1]/div[3]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/ul/li[{i}]/a').get_attribute('href')
            single_url = r.html.find(f'#app-wrapper > div.container--297Nx.all > div:nth-child(3) > div > div.row--3Vhjr.main-container--28aG5.justify-content-flex-start--1Xozy.align-items-normal--vaTgD.flex-wrap-wrap--2PCx8.flex-direction-row--27fh1.flex--3fKk1 > div.sm-col-12--30zDS.lg-col-9--20qCf.block--3v-Ow > div.ad-list-container--1UnyA > div > div.ad-list--2Y3ql > div.list-wrapper--t_A02 > ul > li:nth-child({i}) > a')
            single_url = single_url[0].attrs['href']
            single_page_urls.append(single_url)
            url_cnt += 1
            print("Page: " + str(page_cnt) + " URL: " + str(url_cnt))
        except:
            print('Not details link')
    
    clear_output(wait=True)
print(time.time()-start)
# driver.maximize_window()

241.41733026504517


<IPython.core.display.Javascript object>

In [40]:
len(single_page_urls)

2700

In [39]:
with open(f'category_url/{category}.txt', 'w') as f:
    for s in single_page_urls:
        f.write(s)
        f.write('\n')

In [46]:
def features_collection(text):
    data_dic = {}
    ad_data = text["adDetail"]["data"]["ad"]
    title = ad_data["title"]
    description = ad_data["description"]
    properties = ad_data["properties"]
    features = {}
    for p in properties:
        features[p["label"]] = p["value"]
#     print('Properties', properties)
#     print('Features', features)
    is_member = ad_data["isMember"]
    is_auth_dealer = ad_data["isAuthDealer"]
    is_featured_member = ad_data["isFeaturedMember"]
    membership_level = ad_data["membershipLevel"]
    is_verified = ad_data["isVerified"]
    door_step_delivery = ad_data["isDoorstepDelivery"]
    dsd_applicable = ad_data["isDSDApplicable"]
    timestamp = ad_data["timestamp"]
    date = ad_data["adDate"]
    deactivates = ad_data["deactivates"]
    location_data = ad_data["location"]
    location = location_data["name"]
    geography_data = location_data["geography"]["simple_polygons"][0]["coordinates"]
    geo_polygon_no = len(geography_data)

    category_name = ad_data["category"]["name"]
    area = ad_data["area"]["name"]
    contact_name = ad_data["contactCard"]["name"]
    contact_number = ad_data["contactCard"]["phoneNumbers"]
    phone_number = [p["number"] for p in contact_number]

    shop_name = ad_data["shop"]["name"]
    shop_email = ad_data["shop"]["email"]

    price = ad_data["money"]["amount"]
    delivery_free = ad_data["isDeliveryFree"]

    similar_ads = text["adDetail"]["data"]["similarAds"]

    img_path = []
    image_urls =  text["image"]
    for img_url in image_urls:
        r = requests.get(img_url)
        i = Image.open(BytesIO(r.content))
        img_name = '@'.join(img_url.split('/')[3:5])
        i.save(f"img/{category}/{img_name}.jpg")
        img_path.append(img_name)
    
    data_dic['title'] = title
    data_dic['description'] = description
    data_dic['features'] = features
    data_dic['is_member'] = is_member
    data_dic['is_auth_dealer'] = is_auth_dealer
    data_dic['is_featured_member'] = is_featured_member
    data_dic['membership_level'] = membership_level
    data_dic['is_verified'] = is_verified
    data_dic['door_step_delivery'] = door_step_delivery
    data_dic['dsd_applicable'] = dsd_applicable
    data_dic['timestamp'] = timestamp
    data_dic['date'] = date
    data_dic['deactivates'] = deactivates
    data_dic['geography_data'] = geography_data
    data_dic['geo_polygon_no'] = geo_polygon_no
    data_dic['area'] = area
    data_dic['contact_name'] = contact_name
    data_dic['phone_number'] = phone_number
    data_dic['shop_name'] = shop_name
    data_dic['shop_email'] = shop_email
    data_dic['price'] = price
    data_dic['delivery_free'] = delivery_free
    data_dic['similar_ads'] = similar_ads
    data_dic['image_url'] = image_urls
    data_dic['image_path'] = img_path
    
    return data_dic

In [48]:
category = 'electronics'
session = HTMLSession()

with open(f'category_url/{category}.txt', 'r') as f:
    single_url = f.readlines()
    titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst = [], [], [], [], [], [], []
    cnt = 1
#     print(single_url)
    all_data = []
    for s in single_url[:10]:
        try:
            s = s.replace('\n', '')
            s = 'https://bikroy.com'+s
            print(cnt, s)
            r = session.get('http://localhost:8055/render.html', params={'url': s, 'wait': 5})
            print(r.status_code)       
            text = r.html.text 

            a_split = text.split('window.initialData = ')[1][:-1]
            b_split = a_split.split('"@context":') #"@context": "http://schema.org",
            data = b_split[0][:-3] + ',' + '"@context": ' + b_split[1][:-2]
            data = data.replace('\\', '')
            jtext = json.JSONDecoder().decode(data)

            # attributes extraction
            data_dic = features_collection(jtext)
            data_dic['url'] = s
            all_data.append(data_dic)
    #         if title:
            cnt += 1
            print("=============================================")
#             clear_output(wait=True)
        except:
            print(cnt, s)
            print('Failed')
            print('------------------')
#             pass

1 https://bikroy.com/en/ad/hp-elitebook-core-i7-6th-gen-8gb-ram-and-256gb-ssd-4gb-graphics-for-sale-dhaka-9
200
1 https://bikroy.com/en/ad/hp-elitebook-core-i7-6th-gen-8gb-ram-and-256gb-ssd-4gb-graphics-for-sale-dhaka-9
Failed
------------------
1 https://bikroy.com/en/ad/hp-elitebook-840-g5-corei5-8th-gen-for-sale-chattogram-division
200
1 https://bikroy.com/en/ad/hp-elitebook-840-g5-corei5-8th-gen-for-sale-chattogram-division
Failed
------------------
1 https://bikroy.com/en/ad/sony-bettery-np-fw50-original-for-sale-barishal
200
1 https://bikroy.com/en/ad/sony-bettery-np-fw50-original-for-sale-barishal
Failed
------------------
1 https://bikroy.com/en/ad/aasthaar-aarek-naam-easy-touch-dvr-8-portcamera-6pis1000gb-hdd-for-sale-rajshahi
200
1 https://bikroy.com/en/ad/aasthaar-aarek-naam-easy-touch-dvr-8-portcamera-6pis1000gb-hdd-for-sale-rajshahi
Failed
------------------
1 https://bikroy.com/en/ad/hp-pavilion-15-eh1070wm-amd-ryzen-7512gb-8gb-new-laptop-unbox-for-sale-dhaka
200
1 https:

In [50]:
jtext

{'locale': 'en',
 'marketLocales': ['en', 'bn'],
 'locations': {'type': 'NotAsked'},
 'categories': {'type': 'NotAsked'},
 'category': {'id': 897, 'name': 'Laptop & Computer Accessories'},
 'browserInfo': {'fakeWidth': 1200, 'webpSupported': False},
 'jobsVertical': {'filters': {'type': 'NotAsked'}},
 'serp': {'ads': {'type': 'NotAsked'},
  'staticFilters': {'sort': 'date_desc',
   'posterType': 'all_posters',
   'isDoorstepDelivery': False,
   'isUrgent': False,
   'isFeaturedMember': False,
   'location': -1,
   'category': -1,
   'categorySlug': '',
   'locationSlug': 'bangladesh',
   'page': 1,
   'query': ''},
  'dynamicFilters': {},
  'snapshot': None},
 'verticals': [{'key': 'jobs',
   'name': 'Jobs',
   'categories': [{'id': 700},
    {'id': 1401},
    {'id': 1402},
    {'id': 1404},
    {'id': 1405},
    {'id': 1406},
    {'id': 1407},
    {'id': 1408},
    {'id': 1409},
    {'id': 1410},
    {'id': 1411},
    {'id': 1412},
    {'id': 1413},
    {'id': 1414},
    {'id': 1415},

In [49]:
df = pd.DataFrame(all_data)
df

,title,description,features,is_member,is_auth_dealer,is_featured_member,membership_level,is_verified,door_step_delivery,dsd_applicable,...,contact_name,phone_number,shop_name,shop_email,price,delivery_free,similar_ads,image_url,image_path,url
0,MacBook Air A1466/A1369 & A1465/A1370 Order LCD,Address ;nBranch 1 : Daymon Computer bd nShop ...,"{'Condition': 'New', 'Item type': 'Monitors'}",True,False,False,plus,False,False,True,...,Daymon Computer (BD),"[01722298111, 01744403111]",Daymon Computer(BD),daymon_computer@yahoo.com,"Tk 15,000",False,"[{'id': '62049bd086f565001a90e653', 'slug': 'm...",[https://i.bikroy-st.com/macbook-air-a1466-a13...,[macbook-air-a1466-a1369-and-a1465-a1370-order...,https://bikroy.com/en/ad/macbook-air-a1466-a13...


In [49]:
print(s)

https://bikroy.com/en/ad/oneplus-9-pro-12gb-256gb-global-new-for-sale-chattogram


In [19]:
df_lst = [titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst, single_url[:2]]
column_lst = ['Title', 'Location', 'Price', 'Images', 'Meta Data', 'Feature', 'Description', 'URL']
df = pd.DataFrame (df_lst).T
df.columns = column_lst
df.to_csv(f'csv/{category}.csv', index=False)
df.head()


,Title,Location,Price,Images,Meta Data,Feature,Description,URL
0,Realme C11 (Used),"Posted on 07 Mar 11:15 pm, Maulvi Bazar, Sylhe...","Tk 6,000",[realme-c11-used-for-sale-sylhet-division-12@1...,"{'Condition': 'Used', 'Brand': 'Realme', 'Mode...",N/A,N/A,https://bikroy.com/en/ad/oneplus-9-pro-12gb-25...
1,Samsung Galaxy A7 . (Used),"Posted on 07 Mar 11:15 pm, Dhanmondi, Dhaka","Tk 9,000",[samsung-galaxy-a7-2018-used-for-sale-dhaka-11...,"{'Condition': 'Used', 'Brand': 'Samsung', 'Mod...","Bluetooth, Camera, Dual SIM, Expandable Memory...",N/A,https://bikroy.com/en/ad/oppo-a57-3-32-new-for...


In [20]:
titles

['Realme C11 (Used)', 'Samsung Galaxy A7 . (Used)']